In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/03/03 19:04:59 WARN Utils: Your hostname, Veera resolves to a loopback address: 127.0.1.1; using 172.18.43.37 instead (on interface eth0)
24/03/03 19:04:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 19:05:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 19:05:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2019-10.csv

In [4]:
!wc -l fhv_tripdata_2019-10.csv

1897494 fhv_tripdata_2019-10.csv


In [5]:
df = spark.read \
    .option("header","True") \
    .csv("fhv_tripdata_2019-10.csv")


In [6]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [7]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [8]:
#splitting the large file to small file using 101 rows only to use it in Pandas using linux commands

!head -n 101 fhv_tripdata_2019-10.csv > head.csv

In [9]:
import pandas as pd

In [23]:
df_pandas = pd.read_csv('head.csv')

In [31]:
df_pandas.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264,264,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264,264,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264,264,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264,264,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264,264,NaN,B00014


In [24]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID                int64
DOlocationID                int64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [12]:
# df_pandas.pickup_datetime = pd.to_datetime(df_pandas.pickup_datetime)
# df_pandas.dropOff_datetime = pd.to_datetime(df_pandas.dropOff_datetime)


In [13]:
# import pandas as pd
# pd.DataFrame.iteritems = pd.DataFrame.items

In [25]:
pd.DataFrame.iteritems = pd.DataFrame.items
spark_df = spark.createDataFrame(df_pandas)


/mnt/d/data_eng_zoomcamp/data_eng_ubuntu/spark/spark-3.3.3-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:476: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


TypeError: field Affiliated_base_number: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [28]:
spark_df.schema

StructType([StructField('A', StringType(), True), StructField('B', StringType(), True), StructField('C', DoubleType(), True), StructField('D', DoubleType(), True)])

In [29]:
#downgrade the pandas data frame as attribute error is poping up    
!pip install pandas==1.1.5

#dont know what the issue to downgrade the pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 1.1 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
anceled
ERROR: Operation cancelled by user


In [ ]:
## contuining with the work by copying the schema not using the pandas trick here


In [32]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag ', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [33]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [35]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag : string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [36]:
df = df.repartition(24)

In [37]:
df.write.parquet('fhv/2019/10/')

24/03/03 19:43:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag, Affiliated_base_number
 Schema: dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, SR_Flag , Affiliated_base_number
Expected: SR_Flag  but found: SR_Flag
CSV file: file:///mnt/d/data_zoomcamp_2024/week-5_Batch/pyspark_fhv_tripdata_2019-10/fhv_tripdata_2019-10.csv


24/03/03 19:43:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


24/03/03 19:43:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


24/03/03 19:43:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [38]:
df = spark.read.parquet('fhv/2019/10/')

In [39]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag : string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [40]:
df.count()

1897493

In [41]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+--------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag |Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+--------+----------------------+
|              B02243|2019-10-04 14:09:00|2019-10-04 14:43:00|          56|           7|    null|                LX-038|
|              B01506|2019-10-01 10:41:27|2019-10-01 10:49:35|           7|         226|    null|                B01506|
|              B00821|2019-10-04 07:24:45|2019-10-04 07:26:19|         264|         258|    null|                B00821|
|              B01312|2019-10-01 14:38:14|2019-10-01 14:50:18|         264|          36|    null|                B01312|
|              B01536|2019-10-03 12:24:11|2019-10-03 12:58:59|         264|         243|    null|                B01536|
|              B02715|2019-10-02

In [53]:
from pyspark.sql import functions as F

In [59]:
#add new columns
df.withColumn('pickup_date', F.to_date('pickup_datetime')) \
  .withColumn('dropoff_date', F.to_date('dropoff_datetime')) \
  .withColumn('base_number', F.col('dispatching_base_num')) \
  .select('base_number', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
  .show()

+-----------+-----------+------------+------------+------------+
|base_number|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+-----------+------------+------------+------------+
|     B02243| 2019-10-04|  2019-10-04|          56|           7|
|     B01506| 2019-10-01|  2019-10-01|           7|         226|
|     B00821| 2019-10-04|  2019-10-04|         264|         258|
|     B01312| 2019-10-01|  2019-10-01|         264|          36|
|     B01536| 2019-10-03|  2019-10-03|         264|         243|
|     B02715| 2019-10-02|  2019-10-02|         265|         161|
|     B01280| 2019-10-04|  2019-10-04|         264|         264|
|     B02133| 2019-10-04|  2019-10-04|         264|         264|
|     B01239| 2019-10-02|  2019-10-02|         264|         254|
|     B01338| 2019-10-02|  2019-10-02|         264|          51|
|     B02687| 2019-10-03|  2019-10-03|          67|         123|
|     B00310| 2019-10-01|  2019-10-01|         264|          69|
|     B02546| 2019-10-04|

In [65]:
from pyspark.sql.functions import col

# Assuming 'pickup_datetime' and 'dropoff_datetime' columns are in datetime format
count_on_15th_october = df.filter(
    (col('pickup_datetime').cast('date') == '2019-10-15') |
    (col('dropoff_datetime').cast('date') == '2019-10-15')
).count()

print("Number of records on the 15th of October:", count_on_15th_october)



Number of records on the 15th of October: 63054


In [67]:
df_with_duration = df.withColumn('trip_duration', (col('dropoff_datetime').cast('long') - col('pickup_datetime').cast('long'))/3600)

# Find the longest trip
longest_trip = df_with_duration.orderBy(col('trip_duration').desc()).first()

print("Longest trip details:")
print("Pickup datetime:", longest_trip['pickup_datetime'])
print("Dropoff datetime:", longest_trip['dropoff_datetime'])
print("Trip duration (seconds):", longest_trip['trip_duration'])

Longest trip details:
Pickup datetime: 2019-10-11 18:00:00
Dropoff datetime: 2091-10-11 18:30:00
Trip duration (seconds): 631152.5


In [68]:
lookup = spark.read \
        .option("header","True") \
        .csv('taxi_zone_lookup.csv')

In [72]:
lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [69]:
lookup.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [79]:
#from pyspark.sql.functions import col

# Join FHV October 2019 data with zone lookup data based on 'LocationID' or 'Zone' columns
joined_df = df.join(lookup, df['PULocationID'] == lookup['LocationID'], 'inner')

# Group by 'Zone' and count the occurrences of each zone
pickup_counts = joined_df.groupBy('Zone').count()

# Find the zone with the lowest count
least_frequent_zone = pickup_counts.orderBy(col('count')).first()

print("Name of the least frequent pickup location zone:")
print("Zone:", least_frequent_zone['Zone'])
print("Count:", least_frequent_zone['count'])



Name of the least frequent pickup location zone:
Zone: Jamaica Bay
Count: 1
